# Notebook para generar el dataset

Utilizando la biblioteca Spotipy vamos a generar un csv, haciendo una búsqueda general por año para obtener todas las tracks que fueron publicadas. Para realizar las consultas tomamos el mercado Argentina (AR).
Luego, de cada una de las tracks obtenidas se vovlerá a realizar consultas a la Web API de Spotify para obtener información un poco más técnica de cada una de las tracks.
Con todos esos datos, se generará un DataSet y se lo almacenará en un archivo CSV para su uso posterior.

In [95]:
import spotipy #pip install spotipy --upgrade
from spotipy.oauth2 import SpotifyClientCredentials#para gestionar la autenticacion contra la API de Spotify
import pandas as pd
import numpy as np

# Inicializamos Spotipy

##### Credenciales para conectarse a la web api de Spotify

In [96]:
SPOTIPY_CLIENT_ID = 'e8b2379bdd054eb993035179ae40e066'
SPOTIPY_CLIENT_SECRET = 'f191a3a661cc47d68b92a296669dde05'#este valor puede ser cambiado desde la app

##### Incializamos el objeto sp para poder realizar las requests

In [97]:
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager= client_credentials_manager)

# Realizamos las consultas por año

##### Seteamos los años en que las tracks fueron publicadas

In [98]:
#years = list(range(2000, 2021))#excluye el final
years = [2009]

##### Seteamos la cantidad de máxima de tracks que queremos tomar como muestra para cada uno de los años

In [99]:
max_cantidad_tracks_anio = 10

##### Seteamos un offset inicial para comenzar a tomar desde ahí los tracks

In [100]:
initial_offset = 1000

##### Realizamos la consulta

In [101]:
#declaramos un array de diccionarios donde vamos a ir poniendo los resultados, para luego generar el data frame
data = []

#loopeamos por los años seteados
for year in years:
    print('Año: ' + str(year))    
    
    #configuramos la consulta para el año actual
    search_query = 'year:' + str(year)
    
    #incializamos un contador de tracks, para no traer mas de las que configuramos en max_cantidad_tracks_anio
    tracks_obtenidas = 0        
    
    #definimos una cantidad maxima de reintentos ante errores
    max_retry = 3
    
    #la web api solo permite obtener hasta 50 items a la vez, es por eso que para llegar a cantidad max de tracks
    #por año, se deberá realizar más de una consulta para el mismo año
    while tracks_obtenidas < max_cantidad_tracks_anio:                
        
        #si ya no quedan reintentos, se abandona el año
        if(max_retry<=0):
            print('\nSe consumieron todos los reintentos, se abandona el año.')
            break
        
        try:
            #cuando se hace una consulta a la web api, spotify encuentra por ejemplo 120 resultados
            #pero solo permite traer de a 50 (limit) a la vez, para poder traer todos hay que ir moviéndose en ese
            #resultado con el offset. Para traer las primeras 50, el offset = 0, para traer las siguientes 50, el offset = 50
            result = sp.search(search_query, limit=50, offset=initial_offset+tracks_obtenidas, type='track', market='AR')

            if(result is not None and result['tracks'] is not None and len(result['tracks']['items']) > 0):
                #nos quedamos con las tracks
                tracks_results = list(result['tracks']['items'])

                #por cada uno de los tracks obtenidos, consultamos los datos tecnicos
                #y luego generamos una entrada en el array de diccionarios
                for track_result in tracks_results:
                    track_spotify_id = track_result['id']
                    album = track_result['album']
                    artist = track_result['artists'][0]

                    #nos aseguramos de no agregar mas tracks que las que se establecieron en max_cantidad_tracks_anio
                    if(tracks_obtenidas < max_cantidad_tracks_anio):
                        #buscamos en el array de diccionarios 'data' el artista para saber si ya lo tenemos en memoria o
                        #tendremos que hacer una consulta a la web api para obtener su información
                        datos_artista = [x for x in data if x['artista_id']==artist['id']]
                        
                        if(datos_artista is None or datos_artista == []):
                            #si no se tiene aun el artista se lo va a buscar a spotify
                            artista_web_api_result = sp.artist(artist['id'])
                            datos_artista = {'artista_id': artista_web_api_result['id'], 
                                             'artista_generos': ','.join(artista_web_api_result['genres']),
                                             'artista_followers': artista_web_api_result['followers']['total'] if artista_web_api_result['followers'] is not None and artista_web_api_result['followers'] != [] else -1,
                                             'artista_name': artista_web_api_result['name']}
                        else:
                            #hasta este punto 'datos_artista' es un array de diccionarios, tomamos el primer elemento
                            datos_artista = datos_artista[0]


                        #consultamos a la web api de spotify los datos tecnicos de la track actual                    
                        track_features_result = sp.audio_features(track_spotify_id)

                        if(track_features_result is not None and len(track_features_result) > 0):
                            track_features = track_features_result[0]                    

                            #agregamos los datos al array de diccionarios
                            data.append({'popularity': track_result['popularity'], 'id':track_spotify_id, 
                                         'name':track_result['name'], 'album': album['name'], 'duration_ms':track_result['duration_ms'],
                                         'album_release_date': album['release_date'], 'explicit': track_result['explicit'],
                                         'danceability': track_features['danceability'], 'energy': track_features['energy'],
                                         'key': track_features['key'], 'loudness': track_features['loudness'],
                                         'mode': track_features['mode'], 'speechiness': track_features['speechiness'],
                                         'acousticness': track_features['acousticness'], 'instrumentalness': track_features['instrumentalness'],
                                         'liveness': track_features['liveness'], 'valence': track_features['valence'],
                                         'tempo': track_features['tempo'], 'artista_id': datos_artista['artista_id'],
                                         'artista_generos': datos_artista['artista_generos'], 'artista_followers': datos_artista['artista_followers'],
                                         'artista_name': datos_artista['artista_name'], 'time_signature': track_features['time_signature']
                                        })

                            tracks_obtenidas += 1
                            print(str(tracks_obtenidas) + ' tracks obtenidas!', end='\r')

            else:
                print('Datos vacíos, reintentando')
                #ante cualquier error, bajamos la cantidad de reintentos y seguimos
                max_retry -= 1                    
        
        except Exception as e: 
            print('Error: '+ str(e))
            print('Reintentando')
            #ante cualquier error, bajamos la cantidad de reintentos y seguimos
            max_retry -=1
    print('\nAño completado')
            

Año: 2009
10 tracks obtenidas!
Año completado


# Generamos el DataFrame

In [102]:
df_tracks = pd.DataFrame(data)

In [103]:
df_tracks.shape

(10, 23)

In [104]:
df_tracks.head()

,popularity,id,name,album,duration_ms,album_release_date,explicit,danceability,energy,key,...,acousticness,instrumentalness,liveness,valence,tempo,artista_id,artista_generos,artista_followers,artista_name,time_signature
0,55,3Lkm8fyUksuC8hr9I35nio,Purity,Slipknot (10th Anniversary Edition),265453,2009-09-09,True,0.283,0.978,6,...,0.000105,0.073500,0.2520,0.399,166.178,05fG473iIaoy82BF1aGhL8,"alternative metal,nu metal,rap metal",5797043,Slipknot,4
1,55,7zACUQbWQxzT4aER7ityBV,Poker Face,The Fame Monster (Deluxe Edition),237200,2009-11-05,False,0.851,0.806,4,...,0.118000,0.000002,0.1210,0.787,118.999,1HY2Jd0NmPuamShAr6KMms,"dance pop,pop",12207783,Lady Gaga,4
2,33,4UXaRdu2qwfpdCp7wXAKJ8,The Lady In Red,Cool - Legends,255986,2009-01-01,False,0.561,0.374,10,...,0.363000,0.000016,0.0331,0.488,76.563,2RpHsROrX075xfIwHn6B2U,"mellow gold,new wave pop,soft rock",241614,Chris de Burgh,4
3,59,3mJq2ZlI3ZN8hvmRSdzRy9,Fallin' For You,Breakthrough (Int'l Deluxe Version),215320,2009-01-01,False,0.645,0.825,4,...,0.161000,0.000000,0.1140,0.641,117.037,6aZyMrc4doVtZyKNilOmwu,"acoustic pop,dance pop,neo mellow,pop,pop rock...",1851756,Colbie Caillat,4
4,26,3ngXxVSWacTPQkFhq7pMxo,Y Tu Te Vas - En Vivo,25 Años (En Vivo),143151,2009-04-20,False,0.563,0.850,5,...,0.022200,0.000000,0.7910,0.884,101.741,5L6GbwWUM3Oi5GPnkmmp24,"cumbia pop,cumbia santafesina,cumbia villera",114891,Los del Fuego,4


In [105]:
df_tracks.tail(10)

,popularity,id,name,album,duration_ms,album_release_date,explicit,danceability,energy,key,...,acousticness,instrumentalness,liveness,valence,tempo,artista_id,artista_generos,artista_followers,artista_name,time_signature
0,55,3Lkm8fyUksuC8hr9I35nio,Purity,Slipknot (10th Anniversary Edition),265453,2009-09-09,True,0.283,0.978,6,...,0.000105,0.073500,0.2520,0.399,166.178,05fG473iIaoy82BF1aGhL8,"alternative metal,nu metal,rap metal",5797043,Slipknot,4
1,55,7zACUQbWQxzT4aER7ityBV,Poker Face,The Fame Monster (Deluxe Edition),237200,2009-11-05,False,0.851,0.806,4,...,0.118000,0.000002,0.1210,0.787,118.999,1HY2Jd0NmPuamShAr6KMms,"dance pop,pop",12207783,Lady Gaga,4
2,33,4UXaRdu2qwfpdCp7wXAKJ8,The Lady In Red,Cool - Legends,255986,2009-01-01,False,0.561,0.374,10,...,0.363000,0.000016,0.0331,0.488,76.563,2RpHsROrX075xfIwHn6B2U,"mellow gold,new wave pop,soft rock",241614,Chris de Burgh,4
3,59,3mJq2ZlI3ZN8hvmRSdzRy9,Fallin' For You,Breakthrough (Int'l Deluxe Version),215320,2009-01-01,False,0.645,0.825,4,...,0.161000,0.000000,0.1140,0.641,117.037,6aZyMrc4doVtZyKNilOmwu,"acoustic pop,dance pop,neo mellow,pop,pop rock...",1851756,Colbie Caillat,4
4,26,3ngXxVSWacTPQkFhq7pMxo,Y Tu Te Vas - En Vivo,25 Años (En Vivo),143151,2009-04-20,False,0.563,0.850,5,...,0.022200,0.000000,0.7910,0.884,101.741,5L6GbwWUM3Oi5GPnkmmp24,"cumbia pop,cumbia santafesina,cumbia villera",114891,Los del Fuego,4
5,29,0zSktUOXhFZiZ2IlfrAoSL,Have In Mind,Situations - The Very Best Of,213600,2009-05-01,False,0.613,0.718,6,...,0.065000,0.005050,0.3740,0.776,120.986,0Ii29zDQKo0MN9ULmVgJoI,synthpop,9251,Cetu Javu,4
6,32,6DPJOkWCdll5ES8S8vAhqd,Deja la Vida Volar,Quebrado Vivo,340453,2009,False,0.505,0.562,1,...,0.661000,0.015800,0.9200,0.404,141.420,2FFrhWZS9vJsh2UvxYPRr6,"argentine rock,cantautor,latin alternative,lat...",206008,Pedro Aznar,3
7,65,34sGnIHB3ZthMvHpNX1i7e,Day 'N' Nite - Crookers Remix,Man On The Moon: The End Of Day (Int'l Version),281746,2009-01-01,True,0.756,0.705,11,...,0.032600,0.050300,0.0552,0.962,129.993,0fA0VVWsXO9YnASrzqfmYu,"hip hop,pop rap,rap",3769619,Kid Cudi,4
8,53,4ttCCNTaRSKmgFYOmNQzsO,I Want To Know What Love Is,Memoirs of an imperfect Angel (International V...,207133,2009-01-01,False,0.591,0.420,6,...,0.122000,0.000001,0.1680,0.318,79.957,4iHNK0tOyZPYnBU7nGAgpQ,"dance pop,pop,r&b,urban contemporary",5681187,Mariah Carey,4
9,51,6R1E3mgWYUgXsEIYrRnZoA,Alive,THE E.N.D. (THE ENERGY NEVER DIES) [Deluxe Ver...,303453,2009-01-01,False,0.801,0.643,11,...,0.067600,0.000470,0.6130,0.568,115.038,1yxSLGMDHlW21z4YXirZDS,"dance pop,pop,pop rap",4353578,Black Eyed Peas,4


# Generamos el CSV

In [106]:
tracks_csv_name = 'tracks.csv'

In [107]:
csv_exists = True

In [108]:
#verificamos si el csv existe, si existe lo abrimos para hacer append del dataframe
#si no existe, se crea el archivo
try:
    df_aux = pd.read_csv(tracks_csv_name, header=0)
except:
    csv_exists = False

In [109]:
if(csv_exists):
    df_tracks.to_csv(tracks_csv_name, mode='a', header=False,index=False,encoding='utf-8')
else:
    df_tracks.to_csv(tracks_csv_name,index=False,encoding='utf-8')